<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nicolas Cabello
- Nombre de alumno 2: Esteban Muñoz


### **Link de repositorio de GitHub:** `https://github.com/NicoCabello/Laboratorio_Progra_Cientifica`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

# Clasificadores
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 3.1 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
def mount_drive(path):
    try:
        from google.colab import drive

        drive.mount("/content/drive")
        %cd {path}
    except: 
        print('Ignorando conexión drive-colab')

In [3]:
mount_drive("/content/drive/My Drive/MDS7202/Laboratorio 7/")

Mounted at /content/drive
/content/drive/My Drive/MDS7202/Laboratorio 7


In [4]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [5]:
df_players["Label"] = (~df_players["National_Position"].isnull()).astype(int)

In [6]:
df_players["Label"].value_counts()

0    16513
1     1075
Name: Label, dtype: int64

La clase 0 corresponde a los jugadores que no han sido seleccionados por su equipo nacional, mientras que la clase 1 representa a los jugadores que si han sido seleccionados. Se puede notar que existe un desbalance entre clases, siendo la clase 0 más de 10 veces más grande que la clase 1.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [7]:
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(),  ['Work_Rate']),
        ('OrdinalEncoder', OrdinalEncoder(), ['Weak_foot', 'Skill_Moves']),
        ('StandardScaler', StandardScaler(),['Aggression', 'Reactions', 'Vision', 'Composure', 'Strength', 'Agility', 'Jumping', 'Long_Shots', 'Penalties', 'Volleys']), 
        ('MinMaxScaler', MinMaxScaler(), ['Ball_Control', 'Marking', 'Short_Pass', 'Speed', 'Stamina', 'Heading', 'Finishing'])])

In [8]:
svm_pipe = Pipeline([
    ('preprocessing', preprocessing_transformer),
    ('SVM', SVC())
])

tree_pipe = Pipeline([
    ('preprocessing', preprocessing_transformer),
    ('DecisionTree', DecisionTreeClassifier(criterion='entropy', min_samples_split=3, random_state=42))
])

naive_bayes_pipe = Pipeline([
    ('preprocessing', preprocessing_transformer),
    ('Naive Bayes', GaussianNB())
])

Para la clasificación no todas las variables contenidas en el dataset son relevantes. Por lo mismo, se han omitido las variables *Name*, *Nationality*, *Club_Position* y *Preffered_Foot*. También se han omitido las variables *Height* y *Weight*, pues la información relevante que pueden entregar estas características se encuentran contenidas en las características cuya interpretación es futbolística. Otra característica no tomada en cuenta corresponde a *National_Position*, pues esta feature contiene información sobre la variable a predecir.

En cuánto al resto de las características, se puede notar que existen muchas correlaciones directas entre las características. Por lo mismo, y para evitar un sesgo excesivo respecto a algunos indicadores, se ha decidido seleccionar un subconjunto de todas las características restantes. Por ejemplo, *Marking* está altamente correlacionada con *Sliding_Tackle* y *Standing_Tackle*, por lo que se ha decidido seleccionar sólo la variable *Marking* para realizar la clasificación. Si bien es cierto este método no elimina el problema de la correlación entre variables predictoras, se cree que debe ser capaz de reducir significativamente el sesgo que tendría el modelo en caso de mantener todas las características.

Los clasificadores seleccionados corresponden a un *SVM*, un *Decision Tree* y un *Gaussian Naive Bayes*. La idea detrás de esta selección consiste en contrastar el desempeño de 3 clasificadores con características distintas: por una parte, un SVM y Decision Tree no realizan suposiciones sobre la distribución de las características, mientras que Gaussian Naive Bayes asume que existe independencia entre características, y además que éstas distribuyen respecto a un vector aleatorio gaussiano. Además, SVM y Decision Tree difieren en que un Decision Tree posee una alta varianza en sus predicciones, al punto de que un mismo árbol puede entregar valores muy distintos bajo distintas particiones de un mismo dataset; por otra parte, un Decision Tree puede funcionar con variables categóricas y no necesita estandarizaciones ni normalizaciones, mientras que SVM funciona mejor cuando las variables se encuentran en el mismo rango de valores.

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [9]:
features = df_players.drop('Label',axis=1)
labels = df_players['Label']

X_train, X_test, y_train, y_test = train_test_split(
    features,
    labels,
    test_size=0.33,
    stratify = labels,
    shuffle = True,
    random_state=42)

In [10]:
svm_pipe.fit(X_train, y_train)
tree_pipe.fit(X_train, y_train)
naive_bayes_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(),
                                                  ['Work_Rate']),
                                                 ('OrdinalEncoder',
                                                  OrdinalEncoder(),
                                                  ['Weak_foot', 'Skill_Moves']),
                                                 ('StandardScaler',
                                                  StandardScaler(),
                                                  ['Aggression', 'Reactions',
                                                   'Vision', 'Composure',
                                                   'Strength', 'Agility',
                                                   'Jumping', 'Long_Shots',
                                                   'Penalties', 'Volleys']),
                                  

In [11]:
svm_pred = svm_pipe.predict(X_test)
tree_pred = svm_pipe.predict(X_test)
naive_bayes_pred = svm_pipe.predict(X_test)

In [12]:
print(classification_report(y_test, svm_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5450
           1       0.00      0.00      0.00       355

    accuracy                           0.94      5805
   macro avg       0.47      0.50      0.48      5805
weighted avg       0.88      0.94      0.91      5805



In [13]:
print(classification_report(y_test, tree_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5450
           1       0.00      0.00      0.00       355

    accuracy                           0.94      5805
   macro avg       0.47      0.50      0.48      5805
weighted avg       0.88      0.94      0.91      5805



In [14]:
print(classification_report(y_test, naive_bayes_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5450
           1       0.00      0.00      0.00       355

    accuracy                           0.94      5805
   macro avg       0.47      0.50      0.48      5805
weighted avg       0.88      0.94      0.91      5805



**Respuesta**:

Ninguno de los clasificadores consigue un buen desempeño. Si bien es cierto el accuracy es alto en todos los casos, se debe a que se ha clasificado correctamente la clase mayoritaria, que posee más del 93% de los datos. Por otro lado, si se observa la clase minoritaria, absolutamente ningún dato se ha clasificado de forma correcta.

La métrica *Precision* sirve como indicador del error tipo I, pues toma en cuenta los falsos positivos en su computación; similarmente, *Recall* sirve como un indicador de error tipo II. Cuando se quiere asegurar que no existe un valor bajo en ninguna de estas 2 métricas (o visto de otra forma, no exista un valor muy grande de ninguno de los 2 tipos de errores) se utiliza *F1-score*.

En este ejemplo en particular, no se pueden obtener conclusiones sobre estas métricas pues no existen datos correctamente clasificados para la clase 1. El principal motivo de este resultado se debe a la cantidad de ejemplos por clase, pues el dataset se encuentra muy desbalanceado. Para mejorar el desempeño de los clasificadores se pueden utilizar técnias de resampleo, como oversampling o subsamplig.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [15]:
club_position_map = {
    "ataque" : ['ST', 'CF'] ,
    "central_ataque" : ['RW', 'CAM', 'LW'] ,
    "central" : ['RM', 'CM', 'LM'] ,
    "central_defensa" : ['RWB', 'CDM', 'LWB'],
    "defensa" : ['RB', 'CB', 'LB'],
    "arquero" : ['GK']
    }

new_positions = {v: k for (k, values) in club_position_map.items() for v in values}

modified_df = df_players.copy().drop('Label',axis=1)
modified_df["Modified_Position"] = modified_df["Club_Position"].map(new_positions)
modified_df = modified_df.dropna(subset=['Modified_Position'])

In [16]:
modified_df["Modified_Position"].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Modified_Position, dtype: int64

In [17]:
features = modified_df.drop('Modified_Position',axis=1)
labels = modified_df['Modified_Position']

X_train, X_test, y_train, y_test = train_test_split(
    features,
    labels,
    test_size=0.33,
    stratify = labels,
    shuffle = True,
    random_state=42)

In [18]:
svm_pipe.fit(X_train, y_train)
svm_pred = svm_pipe.predict(X_test)

In [19]:
print(classification_report(y_test, svm_pred))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.75      0.77      0.76       142
        central       0.50      0.63      0.56       299
 central_ataque       0.38      0.14      0.20       192
central_defensa       0.40      0.03      0.05        69
        defensa       0.76      0.96      0.85       389

       accuracy                           0.70      1300
      macro avg       0.63      0.59      0.57      1300
   weighted avg       0.66      0.70      0.66      1300



**Respuesta**:

Dado que los datos de entrenamiento siguen siendo los mismos que en el problema anterior, tanto la selección de características como la elección del clasificador se mantienen para este problema.

La calidad de la clasificación varía de acuerdo a la clase. Por ejemplo, el modelo acierta en todos los ejemplos de conjunto de test cuando se determina que un jugador será buen arquero; similarmente, la clase defensa obtiene métricas sobre el 76%, llegando a un 85% de f1-score. Por otra parte, la clase central_defensa es la que peores métricas entrega, pues el valor más alto corresponde al precision con un 40%, y el más bajo al recall con un 3%. Esto último quiere decir que el modelo tiene un alta cantidad de falsos negativos, por lo que muchos futbolistas cuya mejor posición es la defensa son clasificador en otra categoría.

A pesar de las clases con mal desempeño, en términos generales este clasificador tiene un rendimiento aceptable. Es capaz de discernir entre algunas de las posiciones con relativa facilidad. Posiblemente para las clases con peor desempeño no se tengan las características apropiadas para lograr una buena predicción, por lo que se puede trabajar en el feature engineering para obtener mejores resultados.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [20]:
from sklearn.svm import SVR
from sklearn.metrics import r2_score

In [21]:
sueldos = pd.read_csv('sueldos.csv')
sueldos = sueldos.drop("Unnamed: 0", axis=1).rename({"Player": "Name"}, axis=1)
sueldos

,Name,Weekly Salary
0,Lionel Messi,1538905.0
1,Cristiano Ronaldo,1248536.0
2,Antoine Griezmann,997032.0
3,Neymar,797726.0
4,Gareth Bale,607706.0
...,...,...
2529,Simone Ghidotti,283.0
2530,Valentin Gendrey,100.0
2531,John Yeboah,70.0
2532,Antonis Aidonis,70.0


In [22]:
df_with_salary = df_players.join(sueldos.set_index("Name"), on="Name")
df_with_salary = df_with_salary.dropna(subset=['Weekly Salary'])
df_with_salary

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Label,Weekly Salary
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,85,92,93,90,81,76,85,88,1,1248536.0
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,71,85,95,88,89,90,74,85,1,1538905.0
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,62,78,89,77,79,84,81,83,1,797726.0
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,77,87,94,86,86,84,85,88,1,508923.0
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,25,25,13,16,14,11,47,11,1,326233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16888,Phillip Menzel,Germany,NaN,Res,191,83,Right,18,Medium / Medium,3,...,10,25,5,10,12,8,15,5,0,2034.0
16978,Manuel Akanji,Switzerland,NaN,Sub,187,85,Right,21,Medium / Medium,2,...,51,38,25,18,26,27,40,31,0,54176.0
17105,Moritz Nicolas,Germany,NaN,Res,195,87,Right,19,Medium / Medium,2,...,16,22,9,10,17,14,17,11,0,2262.0
17402,Giacomo Satalino,Italy,NaN,Sub,188,70,Right,17,Medium / Medium,1,...,11,19,6,5,13,11,18,9,0,2827.0


In [23]:
from sklearn.ensemble import RandomForestRegressor

In [39]:
rf = Pipeline([
    ('preprocessing', preprocessing_transformer),
    ('Random Forest', RandomForestRegressor(random_state=42))
])

In [25]:
svr_pipe = Pipeline([
    ('preprocessing', preprocessing_transformer),
    ('SVR', SVR(C=10, kernel='rbf'))
])

In [40]:
features = df_with_salary.drop('Weekly Salary',axis=1)
labels = df_with_salary['Weekly Salary']

X_train, X_test, y_train, y_test = train_test_split(
    features,
    labels,
    test_size=0.33,
    shuffle = True,
    random_state=42)

In [41]:
rf.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(),
                                                  ['Work_Rate']),
                                                 ('OrdinalEncoder',
                                                  OrdinalEncoder(),
                                                  ['Weak_foot', 'Skill_Moves']),
                                                 ('StandardScaler',
                                                  StandardScaler(),
                                                  ['Aggression', 'Reactions',
                                                   'Vision', 'Composure',
                                                   'Strength', 'Agility',
                                                   'Jumping', 'Long_Shots',
                                                   'Penalties', 'Volleys']),
                                  

In [42]:
rf_pred = rf.predict(X_test)
r2_score(y_test, rf_pred)

0.401974387895666

**Respuesta**:

La métrica $R^2$ es un coeficiente que mide la proporción de la varianza de la variable dependiente (en este caso, el sueldo de un jugador) que puede ser explicada por las variables independientes. Se puede interpretar así, como qué tan bie se ajustan los datos al modelo de regresión.

Para este problema se siguen utilizando las características seleccionadas previamente. En cuánto al regresor, se ha utilizado un Random Forest Regressor, el cual consiste de un ensamble de árboles de decisión.

El motivo de esta elección se debe a que otros regresores no alcanzaban el umbral de $R^2$ de 0.35. Es por esto que se ha optado por implementar un regresor más robusto, pues un ensamble de árboles mantiene el bajo sesgo de estos regresores, y permite reducir la varianza de los mismos. Además, un Random Forest no realiza suposiciones sobre la distribcuión de los datos, por lo que se esperaba un mejor rendimiento en comparación a otros regresores.

Es posible que se pueda mejorar el resultado de este regresor si se trabajan mejor las características. Se ha visto desde un inicio que existen muchas correlaciones entre características, por lo que es muy fácil sesgar las predicciones hacia ciertas caratcerísticas en lugar de obtener variables con alto poder predictivo. De aquí, se cree que combinar características, o descorrelacionar/utilizar componentes principales puede ser un buen comienzo para lograr entrenar mejores modelos de aprendizaje supervisado. Por otra parte, para los modelos de Machine Learning siempre es bueno contar con la mayor cantidad de datos posibles, lo que además se vuelve necesario cuando se cuenta con muchas características en el dataset.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>